In [11]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,accuracy_score, mean_squared_error,mean_absolute_percentage_error,r2_score
import tensorflow as tf
from tensorflow import keras
from sklearn.pipeline import make_pipeline

# TESTING on TECH stock

In [12]:
data=yf.download('0700.hk')

[*********************100%***********************]  1 of 1 completed


In [13]:
data=data[["Open","High","Low","Adj Close","Close"]]
data['Prediction']=data['Close'].shift(-1)
data.dropna(inplace=True)
data_val=data[-100:]
data=data[:-100]
data_val.tail()

,Open,High,Low,Adj Close,Close,Prediction
Date,,,,,,
2021-12-24,460.799988,460.799988,460.799988,460.799988,460.799988,450.399994
2021-12-28,457.799988,459.200012,445.000000,450.399994,450.399994,444.799988
2021-12-29,443.600006,445.600006,440.399994,444.799988,444.799988,443.399994
2021-12-30,446.399994,450.799988,442.000000,443.399994,443.399994,458.000000
2021-12-31,458.000000,458.000000,458.000000,458.000000,458.000000,453.799988


In [14]:
sc = MinMaxScaler(feature_range = (-1, 1))

In [16]:
model_tech = keras.Sequential()
model_tech.add(keras.layers.LSTM(
  units=128,
  input_shape=(X_train.shape[1], X_train.shape[2]),
    return_sequences=True
))
model_tech.add(keras.layers.LSTM(units=64))
model_tech.add(keras.layers.Dense(units=64))
model_tech.add(keras.layers.Dense(units=1))
model_tech.compile(
  loss='mean_squared_error',
  optimizer=keras.optimizers.Adam(0.001)
)

In [17]:
model_tech.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_2 (LSTM)               (None, 1, 128)            68608     
                                                                 
 lstm_3 (LSTM)               (None, 64)                49408     
                                                                 
 dense_2 (Dense)             (None, 64)                4160      
                                                                 
 dense_3 (Dense)             (None, 1)                 65        
                                                                 
Total params: 122,241
Trainable params: 122,241
Non-trainable params: 0
_________________________________________________________________


In [ ]:
history_tech=model_tech.fit(
    X_train, y_train,
    epochs=500,
    batch_size=5,
    validation_split=0.1,
    verbose=1,
    shuffle=False
)
y_pred = model_tech.predict(X_test)

Epoch 1/500


2022-01-03 22:19:55.078505: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-01-03 22:19:55.984765: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


In [ ]:
plt.plot(history_tech.history['loss'], label='train')
plt.plot(history_tech.history['val_loss'], label='test')
plt.legend()

In [ ]:
plt.plot(np.arange(0, len(y_train)), y_train, 'g', label="history")
plt.plot(np.arange(len(y_train), len(y_train) + len(y_test)), y_test, marker='.', label="true")
plt.plot(np.arange(len(y_train), len(y_train) + len(y_test)), y_pred, 'r', label="prediction")
plt.ylabel('Value')
plt.xlabel('Time Step')
plt.legend()
plt.show();

In [ ]:
# y_test = sc.inverse_transform(y_test)
# y_pred = sc.inverse_transform(y_pred)
plt.plot(y_test, marker='.', label="true")
plt.plot(y_pred, 'r', label="prediction")
plt.ylabel('Value')
plt.xlabel('Time Step')
plt.legend()
plt.show();

In [ ]:
r2_score(y_test,y_pred)

In [ ]:
mean_absolute_percentage_error(y_test,y_pred)

In [ ]:
mean_squared_error(y_test,y_pred,squared=False)

In [ ]:
x_val=data_val.drop('Prediction',1)
y_val=data_val['Prediction']
x_val=sc.transform(x_val).reshape(x_val.shape[0],1,x_val.shape[1])
y_val_pred=model_tech.predict(x_val)

In [ ]:
plt.plot(y_val.values, marker='.', label="true")
plt.plot(y_val_pred, 'r', marker='*',label="prediction")
plt.ylabel('Value')
plt.xlabel('Time Step')
plt.legend()
plt.show();

In [ ]:
r2_score(y_val,y_val_pred)

In [ ]:
mean_absolute_percentage_error(y_val,y_val_pred)

In [ ]:
mean_squared_error(y_val.values,y_val_pred,squared=False)